Files:
* tn.movie_budgets - production budget, domestic gross, worldwide gross
* tmdb.movies - genre, popularity
* imdb.title.basics - year, runtime, genre

Process Notes:
* Chose to leverage SQLite database - merging dataframes would be difficult because there is not a common unique identifier across all 3 of my data sources

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

In [2]:
def clean_money_columns(df, column):
    df[column] = df[column].apply([lambda x: x[1:]])
    df[column] = df[column].apply([lambda x: x.replace(',', '')])
    df[column] = df[column].apply([lambda x: int(x)])

In [3]:
df_moviebudgets = pd.DataFrame(pd.read_csv('data/tn.movie_budgets.csv'))
df_moviebudgets.isna().any() # checked to make sure no null values
df_moviebudgets.duplicated('movie') # checked for duplicate rows based on "movie" column
df_moviebudgets.drop(columns='id', inplace=True) # dropped ID column because there were duplicate values
df_moviebudgets['release_year'] = df_moviebudgets['release_date'].apply(lambda x: x[-4:]) # created a column for release year to simplify/group
df_moviebudgets.sort_values('release_year', inplace=True)
clean_money_columns(df=df_moviebudgets, column='production_budget')
clean_money_columns(df=df_moviebudgets, column='domestic_gross')
clean_money_columns(df=df_moviebudgets, column='worldwide_gross')
df_moviebudgets['release_year'] = df_moviebudgets['release_year'].astype(int)

print(df_moviebudgets.info())
df_moviebudgets

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 5677 to 535
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   int64 
 3   domestic_gross     5782 non-null   int64 
 4   worldwide_gross    5782 non-null   int64 
 5   release_year       5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 316.2+ KB
None


,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year
5677,"Feb 8, 1915",The Birth of a Nation,110000,10000000,11000000,1915
5523,"Sep 5, 1916",Intolerance,385907,0,0,1916
5614,"Dec 24, 1916","20,000 Leagues Under the Sea",200000,8000000,8000000,1916
5683,"Sep 17, 1920",Over the Hill to the Poorhouse,100000,3000000,3000000,1920
4569,"Dec 30, 1925",Ben-Hur: A Tale of the Christ,3900000,9000000,9000000,1925
...,...,...,...,...,...,...
2292,"Jan 25, 2019",Serenity,25000000,8547045,11367029,2019
4135,"Feb 8, 2019",The Prodigy,6000000,14856291,19789712,2019
1205,"Dec 31, 2020",Hannibal the Conqueror,50000000,0,0,2020
194,"Dec 31, 2020",Moonfall,150000000,0,0,2020


In [72]:
df_imdbbasics = pd.DataFrame(pd.read_csv('data/imdb.title.basics.csv'))
df_imdbbasics.drop(columns='tconst', inplace=True)
df_imdbbasics['genres'] = df_imdbbasics['genres'].fillna('n/a')
df_imdbbasics['original_title'] = df_imdbbasics['original_title'].fillna(df_imdbbasics['primary_title'])
df_imdbbasics['runtime_minutes'] = df_imdbbasics['runtime_minutes'].fillna('0').astype(int)
df_imdbbasics['genres_list'] = df_imdbbasics['genres'].apply(lambda x: x.split(','))
    
print(df_imdbbasics.info())
df_imdbbasics

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   primary_title    146144 non-null  object
 1   original_title   146144 non-null  object
 2   start_year       146144 non-null  int64 
 3   runtime_minutes  146144 non-null  int64 
 4   genres           146144 non-null  object
 5   genres_list      146144 non-null  object
dtypes: int64(2), object(4)
memory usage: 6.7+ MB
None


,primary_title,original_title,start_year,runtime_minutes,genres,genres_list
0,Sunghursh,Sunghursh,2013,175,"Action,Crime,Drama","[Action, Crime, Drama]"
1,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114,"Biography,Drama","[Biography, Drama]"
2,The Other Side of the Wind,The Other Side of the Wind,2018,122,Drama,[Drama]
3,Sabse Bada Sukh,Sabse Bada Sukh,2018,0,"Comedy,Drama","[Comedy, Drama]"
4,The Wandering Soap Opera,La Telenovela Errante,2017,80,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
...,...,...,...,...,...,...
146139,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,[Drama]
146140,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,0,Documentary,[Documentary]
146141,Dankyavar Danka,Dankyavar Danka,2013,0,Comedy,[Comedy]
146142,6 Gunn,6 Gunn,2017,116,n/a,[n/a]


In [80]:
genres_df = df_imdbbasics.loc[df_imdbbasics['start_year']<=2020].sort_values('start_year').drop(columns='genres')

In [98]:
genres_budgets_joined_df = genres_df.set_index('primary_title').join(df_moviebudgets.set_index('movie'), 
                                                                     how='inner')
genres_budgets_joined_df = genres_budgets_joined_df.reset_index().drop(columns=['start_year', 'release_date']).sort_values('worldwide_gross', ascending=False)
genres_budgets_joined_df = genres_budgets_joined_df.loc[genres_budgets_joined_df['worldwide_gross']>0]
exploded_df = genres_budgets_joined_df.explode('genres_list').sort_values('index')
exploded_df

,index,original_title,runtime_minutes,genres_list,production_budget,domestic_gross,worldwide_gross,release_year
1,10 Cloverfield Lane,10 Cloverfield Lane,103,Mystery,5000000,72082999,108286422,2016
1,10 Cloverfield Lane,10 Cloverfield Lane,103,Horror,5000000,72082999,108286422,2016
1,10 Cloverfield Lane,10 Cloverfield Lane,103,Drama,5000000,72082999,108286422,2016
2,10 Days in a Madhouse,10 Days in a Madhouse,111,Drama,12000000,14616,14616,2015
3,12 Rounds,12 Rounds,0,Romance,20000000,12234694,17306648,2009
...,...,...,...,...,...,...,...,...
3807,Zulu,Zulu,110,Drama,16000000,0,1844228,2013
3807,Zulu,Zulu,110,Crime,16000000,0,1844228,2013
3808,xXx: Return of Xander Cage,xXx: Return of Xander Cage,107,Action,85000000,44898413,345033359,2017
3808,xXx: Return of Xander Cage,xXx: Return of Xander Cage,107,Thriller,85000000,44898413,345033359,2017


In [121]:
ax3 = px.box(exploded_df.loc[exploded_df['worldwide_gross']<1500000000], y='genres_list', x='worldwide_gross', 
                             color='genres_list', notched=True,
                             labels={
                                     'worldwide_gross': 'Worldwide Revenue ($B)', 
                                     'genres_list': 'Genre'})

ax3.update_layout(height=800, plot_bgcolor='#f2f2f2')


In [105]:
average_genre_performance = exploded_df.groupby('genres_list').mean().drop(columns=['release_year']).round()
average_genre_performance

,runtime_minutes,production_budget,domestic_gross,worldwide_gross
genres_list,,,,
Action,102.0,66968700.0,72093988.0,191682059.0
Adventure,104.0,92446342.0,108350616.0,301115533.0
Animation,84.0,84884618.0,119075767.0,320676033.0
Biography,100.0,26291726.0,35499057.0,72507253.0
Comedy,95.0,35588685.0,50346253.0,110554497.0
Crime,101.0,29378826.0,32758059.0,70445134.0
Documentary,57.0,27802631.0,34093791.0,71622682.0
Drama,95.0,25434887.0,31058587.0,64995228.0
Family,86.0,50637170.0,73481929.0,171995636.0


In [ ]:
# fig2 = px.bar(top_50.sort_values('worldwide_gross'), y='b_title', x='worldwide_gross', orientation='h',
#             title='Top 50 Highest Grossing Movies', color='release_year', color_continuous_scale='dense',
#             hover_data=['b_title', 'release_year', 'worldwide_gross', 'production_budget', 'genres'],
#             labels={
#                 'worldwide_gross': 'Worldwide Revenue ($B)', 
#                 'release_year': 'Year Released',
#                 'b_title': 'Movie'})
# fig2.update_layout(font=dict(size=10), height=500, plot_bgcolor='#f2f2f2')

# fig2.show()

In [69]:
df_tmdbmovies = pd.DataFrame(pd.read_csv('data/tmdb.movies.csv'))
df_tmdbmovies.isna().any()
df_tmdbmovies.drop(columns=['Unnamed: 0','genre_ids', 'id'], inplace=True)

print(df_tmdbmovies.info())
df_tmdbmovies

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_language  26517 non-null  object 
 1   original_title     26517 non-null  object 
 2   popularity         26517 non-null  float64
 3   release_date       26517 non-null  object 
 4   title              26517 non-null  object 
 5   vote_average       26517 non-null  float64
 6   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 1.4+ MB
None


,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...
26512,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [73]:
import sqlite3
conn = sqlite3.connect('movies.db')
cur = conn.cursor()


df_moviebudgets.to_sql('budgets', con=conn, if_exists='replace', index=True)
df_imdbbasics.drop(columns='genres_list').to_sql('genres', con=conn, if_exists='replace', index=True)
df_tmdbmovies.to_sql('ratings', con=conn, if_exists='replace', index=True)

In [7]:
def query_to_df(query):
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall())
    df.columns = [x[0] for x in cur.description]
    return df

In [8]:
df_tmdbmovies['vote_count'].describe()

count    26517.000000
mean       194.224837
std        960.961095
min          1.000000
25%          2.000000
50%          5.000000
75%         28.000000
max      22186.000000
Name: vote_count, dtype: float64

In [45]:
query1 = '''SELECT b.movie AS b_title, r.title AS r_title, g.primary_title AS g_title, 
                    b.release_year, r.original_language, b.production_budget, b.worldwide_gross, r.vote_average, 
                    r.vote_count, r.popularity, g.runtime_minutes, g.genres
            FROM budgets b
            JOIN ratings r ON b.movie = r.title
            JOIN genres g ON b.movie = g.primary_title
            WHERE b.worldwide_gross > 0
            ORDER BY worldwide_gross DESC
            ;'''
df1 = query_to_df(query1)
joined_df = df1.drop_duplicates(subset=['b_title', 'release_year', 'worldwide_gross'])

joined_df

,b_title,r_title,g_title,release_year,original_language,production_budget,worldwide_gross,vote_average,vote_count,popularity,runtime_minutes,genres
0,Avatar,Avatar,Avatar,2009,en,425000000,2776345279,7.4,18676,26.526,93,Horror
1,Avengers: Infinity War,Avengers: Infinity War,Avengers: Infinity War,2018,en,300000000,2048134200,8.3,13948,80.773,149,"Action, Adventure, Sci-Fi"
2,Jurassic World,Jurassic World,Jurassic World,2015,en,215000000,1648854864,6.6,14056,20.709,124,"Action, Adventure, Sci-Fi"
3,Furious 7,Furious 7,Furious 7,2015,en,190000000,1518722794,7.3,6538,20.396,137,"Action, Crime, Thriller"
4,The Avengers,The Avengers,The Avengers,2012,en,225000000,1517935897,7.6,19673,50.289,143,"Action, Adventure, Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...,...
3962,The Looking Glass,The Looking Glass,The Looking Glass,2015,en,300000,1711,5.0,1,0.600,80,Fantasy
3966,Eddie: The Sleepwalking Cannibal,Eddie: The Sleepwalking Cannibal,Eddie: The Sleepwalking Cannibal,2013,en,1400000,1632,6.2,33,2.586,90,"Comedy, Horror"
3967,Skin Trade,Skin Trade,Skin Trade,2015,en,9000000,1242,5.6,96,9.362,78,Documentary
3969,Higher Power,Higher Power,Higher Power,2018,en,500000,528,4.9,43,7.544,93,"Action, Sci-Fi, Thriller"


In [46]:
top_100 = joined_df.sort_values('worldwide_gross', ascending=False)[0:100]
top_50 = joined_df.sort_values('worldwide_gross', ascending=False)[0:50]

In [48]:
fig = px.bar(top_50.sort_values('worldwide_gross'), y='b_title', x='worldwide_gross', orientation='h',
            title='Top 50 Highest Grossing Movies', color='release_year', color_continuous_scale='dense',
            hover_data=['b_title', 'release_year', 'worldwide_gross', 'production_budget', 'genres'],
            labels={
                'worldwide_gross': 'Worldwide Revenue ($B)', 
                'release_year': 'Year Released',
                'b_title': 'Movie'})
fig.update_layout(font=dict(size=10), height=850, plot_bgcolor='#f2f2f2')

fig.show()

In [123]:
import statsmodels

# Graph depicting movie budgets compared to gross revenue for movies released since 2000, 
# controlled for outliers with revenue > $2B

recent_movie_budgets = df_moviebudgets.loc[(df_moviebudgets['release_year']>2000) & 
                                           (df_moviebudgets['worldwide_gross']<2000000000)]

ax1 = px.scatter(recent_movie_budgets, x='worldwide_gross', y='production_budget', 
                  title='2000-2020 Movies - Revenue Compared to Production Budget',
                  labels={
                      'worldwide_gross': 'Worldwide Revenue ($B)', 
                      'production_budget': 'Production Budget ($M)', 
                      'release_year': 'Year Released'},
                  color='release_year', hover_data=['movie'], trendline='ols', 
                  trendline_color_override='black', color_continuous_scale='dense')
ax1.update_layout(plot_bgcolor='#f2f2f2')
ax1.show()

In [122]:
# Graph depicting movie budgets compared to popularity for movies released since 2000

recent_movie_budgets2 = df2.loc[(df2['release_year']>2000)]

ax2 = px.scatter(recent_movie_budgets2, x='popularity', y='production_budget', 
                  title='2000-2020 Movies - Popularity Compared to Production Budget',
                  labels={
                      'popularity': 'Popularity', 
                      'production_budget': 'Production Budget ($M)', 
                      'release_year': 'Year Released',
                      'b_title': 'Movie'},
                  color='release_year', hover_data=['b_title'], trendline='ols', 
                  trendline_color_override='black', color_continuous_scale='dense')
ax2.update_layout(plot_bgcolor='#f2f2f2')
ax2.show()

In [124]:
# Runtime compared to production budget

ax3 = px.scatter(recent_movie_budgets2, x='runtime_minutes', y='production_budget', 
                  title='2000-2020 Movies - Movie Length Compared to Production Budget',
                  labels={
                      'runtime_minutes': 'Movie Length (minutes)', 
                      'production_budget': 'Production Budget ($M)', 
                      'release_year': 'Year Released',
                      'b_title': 'Movie'},
                  color='release_year', hover_data=['b_title'], trendline='ols', 
                  trendline_color_override='black', color_continuous_scale='dense')
ax3.update_layout(plot_bgcolor='#f2f2f2')
ax3.show()

In [ ]:
#Runtime compared to revenue

In [13]:
recent_movie_budgets.describe().round().drop(columns='release_year')

,production_budget,domestic_gross,worldwide_gross
count,4195.0,4195.0,4.195000e+03
mean,34437633.0,41411983.0,9.662447e+07
std,45306753.0,69232383.0,1.796477e+08
min,1100.0,0.0,0.000000e+00
25%,5000000.0,463928.0,2.479208e+06
50%,18000000.0,14945541.0,2.900506e+07
75%,42000000.0,51577030.0,1.019553e+08
max,410600000.0,700059566.0,1.648855e+09


## code bank


In [ ]:
# code bank

# import json
# f = open('tmdb_genre_ids.json') 
# data = json.load(f)
# df_genreids = pd.DataFrame(data['genres'])
# keys = list(df_genreids['id'])
# values = list(df_genreids['name'])
# genre_dict = dict(zip(keys, values))

# df_tmdbmovies['genre_ids_list'] = df_tmdbmovies['genre_ids'].apply([lambda x: x.strip('[]').split(', ')])

# def str_to_int(str_list):
#     int_list=[]
#     for item in str_list:
#         int_list.append(int(item))
        
# df_tmdbmovies['genre_ids_list'].apply(str_to_int)

# for index in range(len(df_tmdbmovies)):
#     for i in df_tmdbmovies['genre_ids_list'][index]:
#         int(i)

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***